In [27]:
from pyspark import SparkContext
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [4]:
import json

In [21]:
# Initialize SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [22]:
# import dataset
df_filename = "../data/college-search-data-v3.parquet"
df = spark.read.load(df_filename)

In [2]:
MAP_PATH = "./data/college_map.json"

COLLEGEDATA_PATH = "./data/collegedata_general.json"

MAJORLIST_PATH = "./data/collegedata_major_list.json"

INTEGRATED_PATH = "./data/collegedata_integrated.json"

In [11]:
collegedata_general_integrated = {}

major_list = {}

with open(MAJORLIST_PATH) as f:
    major_list = json.load(f)

college_map = {}

with open(MAP_PATH) as f:
    college_map = json.load(f)

valid_name_count = 0
for collegedata_name in college_map.keys():
    if college_map[collegedata_name] != "":
        valid_name_count += 1
print(valid_name_count)

1772


## Integrate major list and clean up collegedata

In [17]:
with open(COLLEGEDATA_PATH) as f:
    collegedata_general = json.load(f)
    for college_name, college_data in collegedata_general.items():
        college_data["admission"].pop("Address", None)
        college_data["admission"].pop("Phone", None)
        college_data["admission"].pop("Overall Admission Rate", None)
        college_data["admission"].pop("Students Enrolled", None)
        college_data["admission"].pop("High School Class Rank", None)

        college_data["financials"].pop("Net Price Calculator", None)
        college_data["financials"].pop("Net Price Calculator", None)

        college_data["academics"].pop("Net Price Calculator", None)

        college_data["campus_life"].pop("Campus Size", None)
        college_data["campus_life"].pop("Nearest Bus Station", None)
        college_data["campus_life"].pop("Nearest Train Station", None)
        college_data["campus_life"].pop("College Housing", None)
        college_data["campus_life"].pop("Types of Housing", None)
        college_data["campus_life"].pop("Housing Requirements", None)
        college_data["campus_life"].pop("Students Living Off Campus/Commuting", None)
        college_data["campus_life"].pop("Off-Campus Housing Assistance", None)

        for campus_life_key in college_data["campus_life"].keys():
            key_word_list = campus_life_key.split(" ")
            if len(key_word_list) >= 2:
                if key_word_list[-1] == "Population":
                    college_data["campus_life"]["City Population"] = college_data["campus_life"].pop(campus_life_key, "Not reported")
                    break

        college_data["students"].pop("Average Starting Salary", None)
        college_data["students"].pop("Disciplines Pursued", None)

        college_data["undergraduate majors"] = major_list[college_name]["undergraduate majors"]

        college_integrate_name = college_map[college_name]
        if college_integrate_name is not None and college_integrate_name != "":
            collegedata_general_integrated[college_integrate_name] = college_data

In [18]:
with open(INTEGRATED_PATH, "w") as f:
    f.write(json.dumps(collegedata_general_integrated, indent=4))

In [19]:
len(collegedata_general_integrated.keys())

1769

## Connect to MongoDB

In [23]:
import pymongo

In [25]:
uri = "mongodb://cledge-db:Tc7PYDfYtB1jBrPfBDPYSNLhlovMKU6AatyKbWPAE75IFy1tLEsczdULMYfaoiFu5bH9qbna704PQCdHBKT8YQ==@cledge-db.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@cledge-db@" # in cledge-db resource, then click Quick Start.
client = pymongo.MongoClient(uri)

db = client["colleges"]
collection = db["colleges-data"]


In [26]:
with open(INTEGRATED_PATH) as f:
    integrated_data = json.load(f)

In [33]:
for key, value in integrated_data.items():
    cur_college = value
    cur_college["college_name"] = key
    cur_college["college_id"] = df.filter(col("INSTNM") == key).select("UNITID").collect()[0][0]
    collection.insert_one(cur_college)

In [39]:
result = collection.find({"college_id": "236948"})

In [ ]:
result[0]